# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [22]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [68]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,talnakh,69.4865,88.3972,21.77,68,99,1.76,RU,1690178400
1,1,sumbe,-11.2061,13.8437,19.56,91,34,0.32,AO,1690178400
2,2,tarawa,1.4190,172.9840,30.74,70,68,4.15,KI,1690178400
3,3,borovsk,55.2064,36.4861,13.41,93,67,2.89,RU,1690178400
4,4,blackmans bay,-43.0167,147.3167,13.23,64,99,4.92,AU,1690178400


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [70]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points(x="Lng",
                                      y="Lat",
                                      frame_height=450,
                                      frame_width=800,
                                      tiles="OSM",
                                      color="City",
                                      size="Humidity",
                                      alpha = 0.5)

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [71]:
#Ideal conditions for me
    #Humidity 65 or higher
    #Max temp should be 16 to 37.7 C
    #Wind speed 5.5 or lower
    #Cloudniness 30 or less

# Narrow down cities that fit criteria and drop any results with null values
indexhumid = city_data_df[(city_data_df['Humidity'] < 65)].index
city_data_df.drop(indexhumid, inplace=True)
indextemp = city_data_df[(city_data_df['Max Temp'] < 16) & (city_data_df['Max Temp'] > 37.7)].index
city_data_df.drop(indextemp, inplace=True)
indexwind = city_data_df[(city_data_df['Wind Speed'] > 5.5)].index
city_data_df.drop(indexwind, inplace=True)
indexcloud = city_data_df[(city_data_df['Cloudiness'] > 30)].index
city_data_df.drop(indexcloud, inplace=True)

# Drop any rows with null values
city_data_df.dropna()

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
7,7,bethel,41.3712,-73.4140,20.33,79,12,0.73,US,1690178400
13,13,chimoio,-19.1164,33.4833,17.67,87,0,2.08,MZ,1690178400
20,20,okhotsk,59.3833,143.3000,19.69,67,11,4.79,RU,1690178400
24,24,ribeira grande,38.5167,-28.7000,22.49,83,28,3.18,PT,1690178400
29,29,vila franca do campo,37.7167,-25.4333,21.27,87,26,2.66,PT,1690178400


### Step 3: Create a new DataFrame called `hotel_df`.

In [72]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City','Country','Lat','Lng','Humidity']].copy()
# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name']=""

# Display sample data
hotel_df.head(20)

,City,Country,Lat,Lng,Humidity,Hotel Name
7,bethel,US,41.3712,-73.4140,79,
13,chimoio,MZ,-19.1164,33.4833,87,
20,okhotsk,RU,59.3833,143.3000,67,
24,ribeira grande,PT,38.5167,-28.7000,83,
29,vila franca do campo,PT,37.7167,-25.4333,87,
34,north bay,CA,46.3168,-79.4663,74,
36,puerto ayora,EC,-0.7393,-90.3518,84,
37,albany,US,42.6001,-73.9662,79,
38,soure,BR,-0.7167,-48.5233,82,
40,ushuaia,AR,-54.8000,-68.3000,83,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [75]:
# Set parameters to search for a hotel
radius = 10000

params = {
    "apiKey":geoapify_key,
    "categories":"accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = hotel_df.loc[index, 'Lat']
    longitude = hotel_df.loc[index, 'Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()

# Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
bethel - nearest hotel: Hampton Inn Danbury
chimoio - nearest hotel: Hotel Amirana
okhotsk - nearest hotel: No hotel found
ribeira grande - nearest hotel: Hospedaria JSF
vila franca do campo - nearest hotel: Vinha d'Areia Beach Hotel
north bay - nearest hotel: Days Inn
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
albany - nearest hotel: No hotel found
soure - nearest hotel: Hotel Marajó
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
tomball - nearest hotel: La Quinta Inn & Suites by Wyndham Tomball
pacific grove - nearest hotel: Pacific Grove Inn
tazovsky - nearest hotel: Тазовчанка
harrison - nearest hotel: Residence Inn
inhambane - nearest hotel: Hotel de Inhambane
palmer - nearest hotel: No hotel found
saldanha - nearest hotel: Hoedjesbaai
thunder bay - nearest hotel: Hampton Inn & Suites by Hilton Thunder Bay
mariehamn - nearest hotel: Hotell Esplanad
celestun - nearest hotel: Hotel Flamingo Playa
punta arenas - nearest hotel: Hote

,City,Country,Lat,Lng,Humidity,Hotel Name
7,bethel,US,41.3712,-73.4140,79,Hampton Inn Danbury
13,chimoio,MZ,-19.1164,33.4833,87,Hotel Amirana
20,okhotsk,RU,59.3833,143.3000,67,No hotel found
24,ribeira grande,PT,38.5167,-28.7000,83,Hospedaria JSF
29,vila franca do campo,PT,37.7167,-25.4333,87,Vinha d'Areia Beach Hotel
...,...,...,...,...,...,...
530,pitimbu,BR,-7.4706,-34.8086,83,Pousada Brisa Mar
534,saint-leu,RE,-21.1500,55.2833,82,Iloha Seaview Hôtel
536,kill devil hills,US,36.0307,-75.6760,80,Mariner Days Inn & Suites
540,nuevitas,CU,21.5453,-77.2644,87,Villa Sabinal (El CECE)


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [76]:
%%capture --no-display

# Configure the map plot
hotel_city_map = hotel_df.hvplot.points(x="Lng",
                                        y="Lat",
                                        frame_height=450,
                                        frame_width=800,
                                        tiles="OSM",
                                        color="City",
                                        size="Humidity",
                                        alpha = 0.5,
                                        hover_cols=["Lng", "Lat", "City", "Humidity", "Hotel Name", "Country"])

# Display the map
hotel_city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)